In [1]:
import pandas as pd
import numpy as np
import sys
sys.path.append('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/src')
from algorithms.battery_optimize import optimize_battery_operation_every_day
from utils.utils import check_missing_timestamps

In [2]:
# configurations for the optimization
max_battery_capacity=2
initial_capacity=1
end_capacity=1
max_battery_power=1
efficiency=0.95
daily_max_charging_circles=100000
safty_fcr_bid_coefficient=1

In [3]:
# read da price data
da_price_data = pd.read_csv('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/processed/day_ahead/da_price_2024.csv')
print(len(check_missing_timestamps(da_price_data, '15min')))
# read fcr price data
fcr_price_data = pd.read_csv('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/processed/balancing_market/fcr/fcr_price_2024.csv')
print(len(check_missing_timestamps(fcr_price_data, '4H')))

Missing timestamps: DatetimeIndex([], dtype='datetime64[ns]', freq='15min')
0
Missing timestamps: DatetimeIndex([], dtype='datetime64[ns]', freq='4h')
0


/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/src/utils/utils.py:35: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  missing_timestamps = complete_timestamps[~complete_timestamps.isin(df['datetime'])]
/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/src/utils/utils.py:34: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  complete_timestamps = pd.date_range(start=df['datetime'].min(), end=df['datetime'].max(), freq=frequence)
/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/src/utils/utils.py:35: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will n

In [4]:
# change fcr_price_data to 15 mins interval
fcr_price_data['datetime'] = pd.to_datetime(fcr_price_data['datetime'])
# Replace '-' with NaN and convert to float
fcr_price_data['GERMANY_SETTLEMENTCAPACITY_PRICE_[EUR/MW]'] = fcr_price_data['GERMANY_SETTLEMENTCAPACITY_PRICE_[EUR/MW]'].replace('-', np.nan).astype(float)
# Resample and forward fill
fcr_price_data = fcr_price_data.resample('15min', on='datetime').last().reset_index()
fcr_price_data = fcr_price_data.fillna(method='ffill')
# Scale the price for 15-minute intervals
fcr_price_data = fcr_price_data[['datetime', 'GERMANY_SETTLEMENTCAPACITY_PRICE_[EUR/MW]']]

/tmp/ipykernel_139132/2826722949.py:7: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  fcr_price_data = fcr_price_data.fillna(method='ffill')


In [5]:
# merge da_price_data and fcr_price_data based on datetime
da_price_data['datetime'] = pd.to_datetime(da_price_data['datetime'])
fcr_price_data['datetime'] = pd.to_datetime(fcr_price_data['datetime'])
mm_price_data = da_price_data.merge(fcr_price_data, on='datetime', how='left')

In [6]:
mm_price_data.head()

,datetime,da_price,GERMANY_SETTLEMENTCAPACITY_PRICE_[EUR/MW]
0,2024-01-01 00:00:00,39.91,114.8
1,2024-01-01 00:15:00,-0.04,114.8
2,2024-01-01 00:30:00,-9.01,114.8
3,2024-01-01 00:45:00,-29.91,114.8
4,2024-01-01 01:00:00,25.28,114.8


In [7]:
def single_day_optimization(mm_price_data, calculate_daily_revenue=True):
    # optimize the battery operation for the every day based on the da price first
    every_day_da_price = mm_price_data.groupby(mm_price_data['datetime'].dt.date)
    result_total = pd.DataFrame()
    for date, single_day_df in every_day_da_price:
        result_single_day = optimize_battery_operation_every_day(
            single_day_df,
            max_battery_capacity,
            initial_capacity,
            end_capacity,
            max_battery_power,
            efficiency,
            daily_max_charging_circles,
            include_revenue=True
            )
        if calculate_daily_revenue:
            result_single_day['total_daily_revenue'] = result_single_day['revenue'].sum()
        # Reset index before concatenation
        result_single_day = result_single_day.reset_index(drop=True)
        # Concatenate with ignore_index=True
        result_total = pd.concat([result_total, result_single_day], ignore_index=True)
    return result_total

def multi_market_optimization(mm_price_data, safty_fcr_bid_coefficient):
    result_total_single_day = single_day_optimization(mm_price_data)
    mm_result = mm_price_data.merge(result_total_single_day, on='datetime', how='left')
    # calculate the revenue for each day
    mm_result['target_fcr_price'] = mm_result['total_daily_revenue'] / 6
    mm_result['scaled_fcr_price'] = mm_result['target_fcr_price'] * safty_fcr_bid_coefficient
    mm_result['success_fcr_bid'] = mm_result['scaled_fcr_price'] >= mm_result['GERMANY_SETTLEMENTCAPACITY_PRICE_[EUR/MW]']
    # if fcr bid failed, do single_day_optimization on failed timeslots of that day
    mm_result_failed = mm_result[mm_result['success_fcr_bid'] == False]
    result_total_failed_fcr_bid = single_day_optimization(mm_result_failed, calculate_daily_revenue=False)
    # add a real_ prefix to every column of result_total_failed_fcr_bid, except for datetime
    result_total_failed_fcr_bid.columns = ['datetime' if col == 'datetime' else 'real_' + col 
                                         for col in result_total_failed_fcr_bid.columns]
    # merge result_total_failed_fcr_bid and mm_result
    result_total = mm_result.merge(result_total_failed_fcr_bid, on='datetime', how='left')
    result_total.loc[result_total.success_fcr_bid == True, 'real_revenue'] = result_total.loc[result_total.success_fcr_bid == True, 'scaled_fcr_price'] / 16
    # ffill real_opening_capacity
    result_total['real_opening_capacity'] = result_total['real_opening_capacity'].ffill()
    return result_total


In [8]:
result_total = multi_market_optimization(mm_price_data, safty_fcr_bid_coefficient)

In [9]:
result_total_slim = result_total[['datetime', 'spot_price', 'GERMANY_SETTLEMENTCAPACITY_PRICE_[EUR/MW]', 'revenue', 'target_fcr_price', 'success_fcr_bid', 'real_power', 'real_market_dispatch', 'real_opening_capacity', 'real_revenue']].rename(columns={'GERMANY_SETTLEMENTCAPACITY_PRICE_[EUR/MW]': 'fcr_price', 'revenue': 'revenue_full_da'} )

In [10]:
result_total_slim.head()

,datetime,spot_price,fcr_price,revenue_full_da,target_fcr_price,success_fcr_bid,real_power,real_market_dispatch,real_opening_capacity,real_revenue
0,2024-01-01 00:00:00,39.91,114.8,9.478625,50.50528,False,1.0,0.2375,1.0000,9.478625
1,2024-01-01 00:15:00,-0.04,114.8,-0.000000,50.50528,False,0.0,0.0000,0.7500,-0.000000
2,2024-01-01 00:30:00,-9.01,114.8,-0.000000,50.50528,False,0.0,0.0000,0.7500,-0.000000
3,2024-01-01 00:45:00,-29.91,114.8,7.477500,50.50528,False,-1.0,-0.2500,0.7500,7.477500
4,2024-01-01 01:00:00,25.28,114.8,6.004000,50.50528,False,1.0,0.2375,0.9875,6.004000


In [11]:
result_total_slim.real_revenue.sum()

np.float64(160658.4527029057)

In [12]:
result_total_slim.to_csv(f'/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/optimization/multi_market_optimization/result_{result_total_slim.datetime.min().date()}_to_{result_total_slim.datetime.max().date()}.csv', index=False)

In [13]:
result_total_slim = pd.read_csv('/home/kehan-vm/Projects/interesting/power_market_analysis/power_market_analysis/data/optimization/multi_market_optimization/result_2024-01-01_to_2024-12-31.csv')

In [14]:
fcr = result_total_slim[result_total_slim['success_fcr_bid'] == True]
spot_market = result_total_slim[result_total_slim['success_fcr_bid'] == False]

In [15]:
print('hours for fcr market is: ', fcr.shape[0]/4)
print('hours for spot market is: ', spot_market.shape[0]/4)

hours for fcr market is:  5288.0
hours for spot market is:  3496.0


In [17]:
result_total_slim.revenue_full_da.sum()

np.float64(150503.15009835528)

In [20]:
circles = result_total_slim[result_total_slim['real_market_dispatch'] > 0].real_market_dispatch.sum()/max_battery_capacity
print('total circles: ', circles)

total circles:  609.4309375
